In [1]:
%run '~/git/Bibliotecas.ipynb'

In [2]:
c=pd.read_csv('~/git/comex_sh_mun/temp/chuvas_m_r_pa_clean.csv')

In [3]:
# Crca a coluna 'date', que servcrá para o controle das bordas da matrcz esparsa
c['date']=pd.to_datetime(c.CO_ANO.astype(str)+'/'+c.CO_MES.astype(str)+'/1')
c.sort_values('date')

,Estação,CO_ANO,CO_MES,mm,Data,date
0,CALDAS,2007,7,118.0,2007-07-01,2007-07-01
177,CONCEICAO DAS ALAGOAS,2007,7,11.0,2007-07-01,2007-07-01
349,PASSA QUATRO,2007,7,62.8,2007-07-01,2007-07-01
350,PASSA QUATRO,2007,9,11.8,2007-09-01,2007-09-01
1,CALDAS,2007,9,11.0,2007-09-01,2007-09-01
...,...,...,...,...,...,...
533,PASSA QUATRO,2023,11,117.8,2023-11-01,2023-11-01
347,CONCEICAO DAS ALAGOAS,2023,11,68.0,2023-11-01,2023-11-01
176,CALDAS,2023,12,96.2,2023-12-01,2023-12-01
348,CONCEICAO DAS ALAGOAS,2023,12,119.8,2023-12-01,2023-12-01


Os procedcmeno a segucr tem objetcvo de, ao transformar a base de dados em matrcz esparsa, não permctcr que as bordas ocupem datas não cobertas pela arqucvo orcgcnal

In [4]:
# Identifica as bordas
c_date_min=c.date.min()
print(c_date_min)
c_date_max=c.date.max()
print(c_date_max)

c_ano_min=c.CO_ANO.min()
c_ano_max=c.CO_ANO.max()

# Crca df com as datas completas
q_date=pd.DataFrame({'CO_ANO':np.arange(c_ano_min,c_ano_max+1),'key':0})\
.merge(pd.DataFrame({'CO_MES':np.arange(1,13),'key':0}),how='outer').drop(columns='key')

q_date['date']=pd.to_datetime(q_date.CO_ANO.astype(str)+'/'+q_date.CO_MES.astype(str)+'/1')

q_date=q_date[(q_date.date>=c_date_min)&(q_date.date<=c_date_max)]
q_date=q_date.date
q_date

2007-07-01 00:00:00
2023-12-01 00:00:00


6     2007-07-01
7     2007-08-01
8     2007-09-01
9     2007-10-01
10    2007-11-01
         ...    
199   2023-08-01
200   2023-09-01
201   2023-10-01
202   2023-11-01
203   2023-12-01
Name: date, Length: 198, dtype: datetime64[ns]

In [5]:
# Mofifica a base de dados para o tipo 'categoria'
c[['Estação','CO_MES']]=c[['Estação','CO_MES']].astype('category')

In [6]:
# Cria  a matriz esparsa da base de dados
c_sprx=c.groupby(by=['CO_ANO','CO_MES','Estação'])['mm'].sum().reset_index()
#c_sprx=c_sprx.set_index('CO_ANO')
c_sprx['date']=pd.to_datetime(c_sprx.CO_ANO.astype(str)+'/'+c_sprx.CO_MES.astype(str)+'/1')
c_s=c_sprx

In [7]:
c_s

,CO_ANO,CO_MES,Estação,mm,date
0,2007,1,CALDAS,0.0,2007-01-01
1,2007,1,CONCEICAO DAS ALAGOAS,0.0,2007-01-01
2,2007,1,PASSA QUATRO,0.0,2007-01-01
3,2007,2,CALDAS,0.0,2007-02-01
4,2007,2,CONCEICAO DAS ALAGOAS,0.0,2007-02-01
...,...,...,...,...,...
607,2023,11,CONCEICAO DAS ALAGOAS,68.0,2023-11-01
608,2023,11,PASSA QUATRO,117.8,2023-11-01
609,2023,12,CALDAS,96.2,2023-12-01
610,2023,12,CONCEICAO DAS ALAGOAS,119.8,2023-12-01


In [8]:
# Limita a matriz esparsa ao quadro de datas
c_s=c_s.merge(q_date)

In [9]:
c_s=c_s.sort_values('date').drop(columns='date')
c_s

,CO_ANO,CO_MES,Estação,mm
0,2007,7,CALDAS,118.0
1,2007,7,CONCEICAO DAS ALAGOAS,11.0
2,2007,7,PASSA QUATRO,62.8
3,2007,8,CALDAS,0.0
4,2007,8,CONCEICAO DAS ALAGOAS,0.0
...,...,...,...,...
588,2023,11,CALDAS,130.0
590,2023,11,PASSA QUATRO,117.8
592,2023,12,CONCEICAO DAS ALAGOAS,119.8
591,2023,12,CALDAS,96.2


In [10]:
c_s[['CO_MES','Estação']]=c_s[['CO_MES','Estação']].astype(str)

In [11]:
c_s['x']=c_s['CO_MES'].str.cat(c_s[['Estação']],sep='-')

In [12]:
k=c_s.groupby('CO_ANO').mm.sum().reset_index().merge(c_s.pivot(index='CO_ANO',columns=['x'],values='mm').reset_index())
k

,CO_ANO,mm,1-CALDAS,1-CONCEICAO DAS ALAGOAS,1-PASSA QUATRO,10-CALDAS,10-CONCEICAO DAS ALAGOAS,10-PASSA QUATRO,11-CALDAS,11-CONCEICAO DAS ALAGOAS,...,6-PASSA QUATRO,7-CALDAS,7-CONCEICAO DAS ALAGOAS,7-PASSA QUATRO,8-CALDAS,8-CONCEICAO DAS ALAGOAS,8-PASSA QUATRO,9-CALDAS,9-CONCEICAO DAS ALAGOAS,9-PASSA QUATRO
0,2007,1536.4,NaN,NaN,NaN,129.3,77.8,89.6,150.4,93.6,...,NaN,118.0,11.0,62.8,0.0,0.0,0.0,11.0,7.0,11.8
1,2008,4212.8,332.6,256.0,254.2,168.0,26.2,105.8,140.6,15.6,...,32.6,0.0,0.0,0.0,35.8,0.0,85.2,63.4,11.6,64.2
2,2009,4350.8,231.2,285.8,194.0,254.6,0.0,210.6,185.4,128.7,...,22.0,21.4,7.2,54.8,81.6,13.4,39.2,144.6,56.2,135.6
3,2010,2727.3,171.6,132.3,151.8,0.0,65.6,84.4,0.0,251.6,...,10.0,4.0,0.0,41.2,0.0,1.6,0.0,0.0,59.0,60.0
4,2011,2462.4,98.0,113.2,415.6,89.3,85.0,77.0,170.6,67.0,...,37.2,0.0,0.0,0.0,0.0,0.0,5.2,0.0,0.0,0.0
5,2012,2899.6,236.8,340.2,117.6,37.2,36.6,43.4,131.6,193.4,...,50.4,25.8,8.4,27.6,0.0,0.0,2.4,27.4,72.2,23.6
6,2013,4069.7,327.8,299.2,320.6,104.8,60.0,122.6,169.2,165.0,...,13.0,30.6,2.8,66.0,3.6,0.0,0.0,59.2,81.8,66.2
7,2014,2563.2,33.0,157.8,25.4,41.0,32.4,32.8,225.8,242.2,...,3.8,23.2,47.2,21.8,0.0,0.0,13.2,45.6,66.8,41.8
8,2015,3416.6,99.0,44.4,85.6,104.8,87.2,84.0,162.2,172.0,...,0.0,2.8,11.4,0.0,3.4,0.0,0.0,140.6,100.8,61.4
9,2016,3685.6,291.2,415.6,324.3,202.6,84.4,95.6,167.4,141.0,...,100.0,0.0,0.0,0.0,15.6,34.6,20.0,2.0,4.2,23.8


In [15]:
k.CO_ANO=k.CO_ANO+1
k

,CO_ANO,mm,1-CALDAS,1-CONCEICAO DAS ALAGOAS,1-PASSA QUATRO,10-CALDAS,10-CONCEICAO DAS ALAGOAS,10-PASSA QUATRO,11-CALDAS,11-CONCEICAO DAS ALAGOAS,...,6-PASSA QUATRO,7-CALDAS,7-CONCEICAO DAS ALAGOAS,7-PASSA QUATRO,8-CALDAS,8-CONCEICAO DAS ALAGOAS,8-PASSA QUATRO,9-CALDAS,9-CONCEICAO DAS ALAGOAS,9-PASSA QUATRO
0,2008,1536.4,NaN,NaN,NaN,129.3,77.8,89.6,150.4,93.6,...,NaN,118.0,11.0,62.8,0.0,0.0,0.0,11.0,7.0,11.8
1,2009,4212.8,332.6,256.0,254.2,168.0,26.2,105.8,140.6,15.6,...,32.6,0.0,0.0,0.0,35.8,0.0,85.2,63.4,11.6,64.2
2,2010,4350.8,231.2,285.8,194.0,254.6,0.0,210.6,185.4,128.7,...,22.0,21.4,7.2,54.8,81.6,13.4,39.2,144.6,56.2,135.6
3,2011,2727.3,171.6,132.3,151.8,0.0,65.6,84.4,0.0,251.6,...,10.0,4.0,0.0,41.2,0.0,1.6,0.0,0.0,59.0,60.0
4,2012,2462.4,98.0,113.2,415.6,89.3,85.0,77.0,170.6,67.0,...,37.2,0.0,0.0,0.0,0.0,0.0,5.2,0.0,0.0,0.0
5,2013,2899.6,236.8,340.2,117.6,37.2,36.6,43.4,131.6,193.4,...,50.4,25.8,8.4,27.6,0.0,0.0,2.4,27.4,72.2,23.6
6,2014,4069.7,327.8,299.2,320.6,104.8,60.0,122.6,169.2,165.0,...,13.0,30.6,2.8,66.0,3.6,0.0,0.0,59.2,81.8,66.2
7,2015,2563.2,33.0,157.8,25.4,41.0,32.4,32.8,225.8,242.2,...,3.8,23.2,47.2,21.8,0.0,0.0,13.2,45.6,66.8,41.8
8,2016,3416.6,99.0,44.4,85.6,104.8,87.2,84.0,162.2,172.0,...,0.0,2.8,11.4,0.0,3.4,0.0,0.0,140.6,100.8,61.4
9,2017,3685.6,291.2,415.6,324.3,202.6,84.4,95.6,167.4,141.0,...,100.0,0.0,0.0,0.0,15.6,34.6,20.0,2.0,4.2,23.8


In [16]:
z=k

In [17]:
z.to_csv('~/git/comex_sh_mun/temp/chuvas_m_r_pa_clean_wide.csv',index=False)
z

,CO_ANO,mm,1-CALDAS,1-CONCEICAO DAS ALAGOAS,1-PASSA QUATRO,10-CALDAS,10-CONCEICAO DAS ALAGOAS,10-PASSA QUATRO,11-CALDAS,11-CONCEICAO DAS ALAGOAS,...,6-PASSA QUATRO,7-CALDAS,7-CONCEICAO DAS ALAGOAS,7-PASSA QUATRO,8-CALDAS,8-CONCEICAO DAS ALAGOAS,8-PASSA QUATRO,9-CALDAS,9-CONCEICAO DAS ALAGOAS,9-PASSA QUATRO
0,2008,1536.4,NaN,NaN,NaN,129.3,77.8,89.6,150.4,93.6,...,NaN,118.0,11.0,62.8,0.0,0.0,0.0,11.0,7.0,11.8
1,2009,4212.8,332.6,256.0,254.2,168.0,26.2,105.8,140.6,15.6,...,32.6,0.0,0.0,0.0,35.8,0.0,85.2,63.4,11.6,64.2
2,2010,4350.8,231.2,285.8,194.0,254.6,0.0,210.6,185.4,128.7,...,22.0,21.4,7.2,54.8,81.6,13.4,39.2,144.6,56.2,135.6
3,2011,2727.3,171.6,132.3,151.8,0.0,65.6,84.4,0.0,251.6,...,10.0,4.0,0.0,41.2,0.0,1.6,0.0,0.0,59.0,60.0
4,2012,2462.4,98.0,113.2,415.6,89.3,85.0,77.0,170.6,67.0,...,37.2,0.0,0.0,0.0,0.0,0.0,5.2,0.0,0.0,0.0
5,2013,2899.6,236.8,340.2,117.6,37.2,36.6,43.4,131.6,193.4,...,50.4,25.8,8.4,27.6,0.0,0.0,2.4,27.4,72.2,23.6
6,2014,4069.7,327.8,299.2,320.6,104.8,60.0,122.6,169.2,165.0,...,13.0,30.6,2.8,66.0,3.6,0.0,0.0,59.2,81.8,66.2
7,2015,2563.2,33.0,157.8,25.4,41.0,32.4,32.8,225.8,242.2,...,3.8,23.2,47.2,21.8,0.0,0.0,13.2,45.6,66.8,41.8
8,2016,3416.6,99.0,44.4,85.6,104.8,87.2,84.0,162.2,172.0,...,0.0,2.8,11.4,0.0,3.4,0.0,0.0,140.6,100.8,61.4
9,2017,3685.6,291.2,415.6,324.3,202.6,84.4,95.6,167.4,141.0,...,100.0,0.0,0.0,0.0,15.6,34.6,20.0,2.0,4.2,23.8
